In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem.wordnet import WordNetLemmatizer
stopwords=set(stopwords.words('romanian'))
from nltk.tokenize import word_tokenize
import gensim
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
nltk.download([
...     "names",
...     "stopwords",
...     "state_union",
...     "twitter_samples",
...     "movie_reviews",
...     "averaged_perceptron_tagger",
...     "vader_lexicon",
...     "punkt",
... ])

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Zoe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_d

True

In [4]:
from collections import defaultdict
from collections import  Counter
import re
import string
plt.style.use('ggplot')

In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
df = pd.read_excel(r'C:\Users\Zoe\Desktop\Disertatie\Coduri\PM_RI cu etichete.xlsx')

In [8]:
df.head()

,Data,Header,Text,RI,RID,Directie
0,2005-01-17,Consiliul de administratie al BNR a hotarat re...,"in sedinta sa din 14-15 ianuarie 2005, Consili...",8.9,-0.4,s
1,2005-02-12,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 10 februarie 2005, Consiliul de...",8.7,-0.2,s
2,2005-03-14,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 12 martie 2005, Consiliul de Ad...",10.0,1.3,c
3,2005-04-08,NaN,"in sedinta sa din 7 aprilie 2005, Consiliul de...",9.3,-0.7,s
4,2005-07-08,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 8 iulie 2005, Consiliul de Admi...",8.9,-0.4,s


In [9]:

df['year'] = pd.DatetimeIndex(df['Data']).year
df['month'] = pd.DatetimeIndex(df['Data']).month
df['month_year'] = pd.to_datetime(df['Data']).dt.to_period('M')
df.head()
df['date'] =  pd.to_datetime(df['Data']).dt.strftime('%Y-%m-%d')

In [10]:
df.head()

,Data,Header,Text,RI,RID,Directie,year,month,month_year,date
0,2005-01-17,Consiliul de administratie al BNR a hotarat re...,"in sedinta sa din 14-15 ianuarie 2005, Consili...",8.9,-0.4,s,2005,1,2005-01,2005-01-17
1,2005-02-12,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 10 februarie 2005, Consiliul de...",8.7,-0.2,s,2005,2,2005-02,2005-02-12
2,2005-03-14,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 12 martie 2005, Consiliul de Ad...",10.0,1.3,c,2005,3,2005-03,2005-03-14
3,2005-04-08,NaN,"in sedinta sa din 7 aprilie 2005, Consiliul de...",9.3,-0.7,s,2005,4,2005-04,2005-04-08
4,2005-07-08,Consiliul de Administratie al Bancii Nationale...,"in sedinta din 8 iulie 2005, Consiliul de Admi...",8.9,-0.4,s,2005,7,2005-07,2005-07-08


In [11]:
def remove_string(data, regex):
    return data.str.lower().str.replace(regex.lower(), ' ')

def cleanDataset(dataset, columnsToClean, regexList):
    for column in columnsToClean:
        for regex in regexList:
            dataset[column] = remove_string(dataset[column], regex)
    return dataset

def get_regex_list():
    '''
    add here expressions you want to filter out
    '''
    regexList = []
    regexList += ['[^a-zA-z 0-9]+']
    regexList += ['[0-9]']
    regexList += ['https?:[^\]\n\r]+']
    regexList += ['http?:[^\]\n\r]+']
    regexList += ['co?:[^\]\n\r]+']
    regexList += ["  "]
    regexList += ['http']
    return regexList

In [12]:
def preprocess(text):
    result = []
    for x in gensim.utils.simple_preprocess(text,min_len=3, max_len=13):
        if x not in stopwords:
            result.append(x)     
    return result

In [13]:
def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word)

In [14]:
def prepare_text(tokens):
    tokens = [get_lemma(token) for token in tokens]
    return ' '.join(tokens)

In [15]:
def sentimentScore(text):
    analyzer = SentimentIntensityAnalyzer()
    results = []
    for sentence in text:
        vs = analyzer.polarity_scores(sentence)
        results.append(vs)
    return results

In [16]:
import time
from sklearn.feature_extraction.text import CountVectorizer


def count_vect_feature(feature, df, max_features=5000):
    start_time = time.time()
    cv = CountVectorizer(max_features=max_features,
                             ngram_range=(1, 1),
                             stop_words=stopwords)
    X_feature = cv.fit_transform(df[feature])
    print('Count Vectorizer `{}` completed in {} sec.'.format(feature, round(time.time() - start_time,2)))
    return X_feature, cv

In [17]:
X_text, cv = count_vect_feature('Text', df)

Count Vectorizer `Text` completed in 0.18 sec.


C:\Users\Zoe\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['printr'] not in stop_words.
  warnings.warn(


In [18]:
regex_list = get_regex_list()
x = cleanDataset(df, ['Text'],regex_list)

C:\Users\Zoe\AppData\Local\Temp\ipykernel_8572\1815370050.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  return data.str.lower().str.replace(regex.lower(), ' ')


In [20]:
df['prep_text'] = x['Text'].apply(lambda y:preprocess(y))
df['prep_text'] = df['prep_text'].apply(lambda y:prepare_text(y))

In [21]:
df.head()

,Data,Header,Text,RI,RID,Directie,year,month,month_year,date,prep_text
0,2005-01-17,Consiliul de administratie al BNR a hotarat re...,in sedinta sa din ianuarie consiliul de ...,8.9,-0.4,s,2005,1,2005-01,2005-01-17,sedinta ianuarie consiliul administratie banci...
1,2005-02-12,Consiliul de Administratie al Bancii Nationale...,in sedinta din februarie consiliul de admi...,8.7,-0.2,s,2005,2,2005-02,2005-02-12,sedinta februarie consiliul administratie banc...
2,2005-03-14,Consiliul de Administratie al Bancii Nationale...,in sedinta din martie consiliul de adminis...,10.0,1.3,c,2005,3,2005-03,2005-03-14,sedinta martie consiliul administratie bancii ...
3,2005-04-08,NaN,in sedinta sa din aprilie consiliul de adm...,9.3,-0.7,s,2005,4,2005-04,2005-04-08,sedinta aprilie consiliul administratie bancii...
4,2005-07-08,Consiliul de Administratie al Bancii Nationale...,in sedinta din iulie consiliul de administ...,8.9,-0.4,s,2005,7,2005-07,2005-07-08,sedinta iulie consiliul administratie bancii n...


In [22]:
def sentimentScore(text):
    analyzer = SentimentIntensityAnalyzer()
    results = []
    for sentence in text:
        vs = analyzer.polarity_scores(sentence)
        results.append(vs)
    return results

In [23]:
df_results = pd.DataFrame(sentimentScore(df['prep_text']))
df=pd.merge(df, df_results, left_index=True, right_index=True)
df.set_index('month_year', inplace=True)

In [24]:
df['polarity']=(df.pos-df.neg)/(df.pos+df.neg)

In [25]:
df = df.fillna(0)

In [73]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
import pydotplus
from six import StringIO  
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score
import xlsxwriter
import os

In [74]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Example corpus
processed_corpus =  np.array(df['prep_text'])

processed_corpus

array(['sedinta ianuarie consiliul administratie bancii nationale romaniei analizat evolutiile recente ansamblului indicatorilor economici perspectivele acestora anul efect temperarii ritmului crestere preturilor consum ultimele doua luni rata anuala inflatiei coborat decembrie nivelul suta incadrandu marja variatie acceptabila obiectivul asumat anul pornind rezultat proiectia termen scurt ratei inflatiei reconfigurata ingloband efectele modificarii ipoteze privind ajustarea preturilor administrate evolutia cursului schimb leului cresterea veniturilor disponibile urmare modificarii cotelor impozitare accelerarea ultimele luni dezinflatiei permis consiliului administratie bancii nationale romaniei decida reducerea ratei dobanzii interventie puncte procentuale pana nivelul suta ultimele luni rata dobanzii interventie redus puncte procentuale scadere puncte procentuale ratei inflatiei totodata consiliul administratie bancii nationale romaniei hotarat nivelului ratei dobanzii facilitatea c

In [75]:
# Train Word2Vec model
model = Word2Vec(processed_corpus, window=5, min_count=1, workers=4)



In [76]:
# Get word embeddings
word_embeddings = model.wv

In [77]:
word_embeddings

In [78]:
x = np.array([np.mean([model.wv[word] for word in doc], axis=0) for doc in processed_corpus])
y = np.array(df['Directie'])


In [123]:
from sklearn.decomposition import PCA
import numpy as np

n_components = 7
pca = PCA(n_components=n_components)

X_pca = pca.fit_transform(x)

# Afișați rezultatele
print("Dimensiunea originală a datelor:", x.shape)
print("Dimensiunea datelor după aplicarea PCA:", X_pca.shape)
print("Componentele principale:", pca.components_)
print("Varianța explicată:", pca.explained_variance_ratio_)


Dimensiunea originală a datelor: (142, 100)
Dimensiunea datelor după aplicarea PCA: (142, 7)
Componentele principale: [[ 7.43381009e-02 -2.52436936e-01 -2.32300878e-01 -8.65814090e-02
   9.47972573e-03 -3.35713923e-02 -2.77813450e-02  5.93555309e-02
   4.89284471e-02  2.42122039e-02 -6.62185326e-02  1.30000114e-01
  -1.67620778e-02  4.60985079e-02 -6.51505068e-02 -7.14602172e-02
  -1.30559877e-01 -1.00462005e-01  1.77999243e-01  2.30047777e-01
   7.82560557e-02 -3.35101560e-02 -2.00509772e-01  2.60069519e-01
   6.62555471e-02 -8.79604071e-02 -3.01660933e-02  5.35013899e-02
  -5.36592528e-02 -1.00857340e-01  1.05498783e-01  6.52353019e-02
  -1.65838059e-02 -1.69654787e-01 -3.62256393e-02  6.78871721e-02
   3.41042541e-02 -2.02113600e-03 -9.48479474e-02 -1.55044392e-01
  -1.24860033e-02 -3.23897153e-02  2.56331842e-02  2.16685068e-02
   7.83929303e-02 -4.02373187e-02 -9.15502086e-02  3.81722562e-02
   1.17631353e-01  4.41871360e-02  2.54776720e-02  6.82736933e-02
  -4.39340658e-02  2.381

In [124]:
X_pca

array([[-1.08186137e-02, -2.65395269e-02,  3.11641279e-03,
        -1.15344962e-02, -1.42887738e-02, -1.93883991e-03,
        -2.71152728e-03],
       [-4.19193320e-03, -1.09641943e-02,  1.11559387e-02,
        -1.65936276e-02,  5.82391582e-03,  3.35891003e-04,
         1.61007196e-02],
       [ 9.36935376e-03, -1.31383752e-02,  2.44731596e-03,
        -1.39987944e-02, -4.12696507e-03,  1.79805737e-02,
        -4.23519319e-04],
       [ 1.00340825e-02, -2.62090154e-02,  6.63362211e-03,
        -7.01247761e-03, -2.06307694e-02,  9.88031738e-03,
        -3.08620511e-05],
       [-4.51521808e-03,  1.95730431e-03, -4.90831817e-03,
         2.19950991e-04,  4.47211415e-03,  4.74402402e-03,
        -9.63130500e-04],
       [ 3.15371305e-02, -4.03085584e-03,  3.98543337e-03,
        -7.69679435e-03, -3.98069667e-03,  1.73516590e-02,
        -1.77227482e-02],
       [ 8.28341115e-03, -8.15154798e-03, -6.06001308e-03,
         5.31468482e-04,  3.51791823e-04,  5.56544121e-03,
        -1.0723832

In [147]:

# from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [148]:
print(x_train.shape)

(106, 100)


### KNN - K-Nearest-Neighbors

In [149]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train) 
neigh.score( x_test, y_test)

C:\Users\Zoe\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.4444444444444444

In [150]:
neigh.score(x_train, y_train)

C:\Users\Zoe\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.7264150943396226

In [151]:
neigh_cv = cross_val_score(neigh, x_train, y_train, cv=10) 
print(neigh_cv.mean()) 

0.610909090909091


C:\Users\Zoe\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Zoe\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` t

Logistic Regression

In [152]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42)
logreg.fit(x_train, y_train)
logreg.score( x_test, y_test)

0.3611111111111111

In [153]:
logreg.score(x_train, y_train)

0.5094339622641509

In [154]:
logreg_cv = cross_val_score(logreg, x_train, y_train, cv=10)
print(logreg_cv.mean())

0.509090909090909


In [155]:
from sklearn.svm import SVC

svm_linear = SVC( kernel = 'linear')
svm_linear.fit(x_train, y_train)
svm_linear.score(x_test, y_test)

0.3611111111111111

In [156]:
svm_linear.score(x_train, y_train)

0.5094339622641509

In [157]:
svm_linear_cv = cross_val_score(svm_linear, x_train, y_train, cv=10)
print(svm_linear_cv.mean())

0.509090909090909


In [158]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
X_minmax = scaler.fit_transform(x_train)
X_minmax_test = scaler.fit_transform(x_test)
mnb = MultinomialNB()
mnb.fit(X_minmax, y_train)

MultinomialNB()

In [159]:
mnb.score(X_minmax_test, y_test)

0.6111111111111112

In [160]:
mnb.score(X_minmax, y_train)

0.6792452830188679

In [161]:
mnb_cv = cross_val_score(mnb, X_minmax, y_train, cv=10) 
print(mnb_cv.mean())

0.5900000000000001


In [162]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier(random_state=42)
dtc.fit(x_train, y_train) 
dtc.score(x_test, y_test)
dtc

DecisionTreeClassifier(random_state=42)

In [163]:
dtc_cv = cross_val_score(dtc, x_train, y_train, cv=10) 
print(dtc_cv.mean())

0.48


In [164]:
forest_reg = RandomForestClassifier(random_state=42)
forest_reg.fit(x_train, y_train)
forest_reg.score(x_test, y_test)

0.4166666666666667

In [165]:
forest_reg.score(x_train, y_train)

1.0

In [166]:
forest_reg_cv = cross_val_score(forest_reg, x_train, y_train, cv=10) 
print(forest_reg_cv.mean())  

0.5036363636363637


In [167]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Create an AdaBoost classifier
adaboost = AdaBoostClassifier(n_estimators=50, random_state=42)

# Train the classifier
adaboost.fit(x_train, y_train)

# Predict on the test set
y_pred = adaboost.predict(x_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4722222222222222


In [168]:
y_pred = adaboost.predict(x_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [169]:
adaboost_reg_cv = cross_val_score(adaboost, x_train, y_train, cv=10) 
print(adaboost_reg_cv.mean()) 

0.5245454545454545


In [170]:
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Convert class labels to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

# Create an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, random_state=52)

# Train the classifier
xgb_classifier.fit(x_train, y_train)

# Predict on the test set
y_pred = xgb_classifier.predict(x_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3888888888888889


In [171]:
y_pred = xgb_classifier.predict(x_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [172]:
xgb_classifier_cv = cross_val_score(xgb_classifier, x_train, y_train, cv=10) 
print(xgb_classifier_cv.mean()) 

0.5445454545454546


In [173]:
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a CatBoost classifier
catboost = CatBoostClassifier(iterations=100, random_state=42)

# Train the classifier
catboost.fit(x_train, y_train)

# Predict on the test set
y_pred = catboost.predict(x_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Learning rate set to 0.032639
0:	learn: 0.6812911	total: 22.9ms	remaining: 2.26s
1:	learn: 0.6650921	total: 37.2ms	remaining: 1.82s
2:	learn: 0.6534826	total: 50.2ms	remaining: 1.62s
3:	learn: 0.6412986	total: 63.1ms	remaining: 1.51s
4:	learn: 0.6336491	total: 75.1ms	remaining: 1.43s
5:	learn: 0.6225827	total: 86.3ms	remaining: 1.35s
6:	learn: 0.6099827	total: 97.6ms	remaining: 1.3s
7:	learn: 0.5995647	total: 109ms	remaining: 1.25s
8:	learn: 0.5918996	total: 120ms	remaining: 1.22s
9:	learn: 0.5789961	total: 131ms	remaining: 1.18s
10:	learn: 0.5705996	total: 142ms	remaining: 1.15s
11:	learn: 0.5641457	total: 153ms	remaining: 1.13s
12:	learn: 0.5546490	total: 165ms	remaining: 1.1s
13:	learn: 0.5455564	total: 176ms	remaining: 1.08s
14:	learn: 0.5335912	total: 188ms	remaining: 1.06s
15:	learn: 0.5210137	total: 201ms	remaining: 1.05s
16:	learn: 0.5099441	total: 218ms	remaining: 1.06s
17:	learn: 0.5016891	total: 229ms	remaining: 1.04s
18:	learn: 0.4954789	total: 242ms	remaining: 1.03s
19:	le

In [122]:
# Predict on the test set
y_pred = catboost.predict(x_train)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy:", accuracy)

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 10 is present in model but not in pool.

In [121]:
catboost_CV = cross_val_score(catboost, x_train, y_train, cv=10) 
print(catboost_CV.mean()) 

Learning rate set to 0.031973
0:	learn: 0.6907899	total: 71.5ms	remaining: 7.08s
1:	learn: 0.6868805	total: 73.1ms	remaining: 3.58s
2:	learn: 0.6842488	total: 74.6ms	remaining: 2.41s
3:	learn: 0.6819018	total: 76.1ms	remaining: 1.82s
4:	learn: 0.6785352	total: 77.6ms	remaining: 1.47s
5:	learn: 0.6757252	total: 79.8ms	remaining: 1.25s
6:	learn: 0.6728488	total: 81.6ms	remaining: 1.08s
7:	learn: 0.6699706	total: 83.1ms	remaining: 955ms
8:	learn: 0.6677965	total: 84.5ms	remaining: 855ms
9:	learn: 0.6651636	total: 86.1ms	remaining: 775ms
10:	learn: 0.6624990	total: 88.4ms	remaining: 715ms
11:	learn: 0.6603838	total: 89.8ms	remaining: 659ms
12:	learn: 0.6571629	total: 91.3ms	remaining: 611ms
13:	learn: 0.6543802	total: 92.8ms	remaining: 570ms
14:	learn: 0.6517251	total: 94.3ms	remaining: 534ms
15:	learn: 0.6489746	total: 96.6ms	remaining: 507ms
16:	learn: 0.6459767	total: 98.1ms	remaining: 479ms
17:	learn: 0.6436282	total: 99.6ms	remaining: 454ms
18:	learn: 0.6405261	total: 102ms	remaining:

Learning rate set to 0.031973
0:	learn: 0.6903051	total: 2.41ms	remaining: 238ms
1:	learn: 0.6882957	total: 4.16ms	remaining: 204ms
2:	learn: 0.6848289	total: 6.28ms	remaining: 203ms
3:	learn: 0.6820226	total: 8.08ms	remaining: 194ms
4:	learn: 0.6791333	total: 10.3ms	remaining: 196ms
5:	learn: 0.6773281	total: 12ms	remaining: 188ms
6:	learn: 0.6742832	total: 13.8ms	remaining: 183ms
7:	learn: 0.6713440	total: 15.6ms	remaining: 179ms
8:	learn: 0.6692731	total: 17.4ms	remaining: 176ms
9:	learn: 0.6671039	total: 19.5ms	remaining: 176ms
10:	learn: 0.6652109	total: 21.8ms	remaining: 176ms
11:	learn: 0.6629067	total: 23.6ms	remaining: 173ms
12:	learn: 0.6610333	total: 25.5ms	remaining: 170ms
13:	learn: 0.6589460	total: 28ms	remaining: 172ms
14:	learn: 0.6565800	total: 29.9ms	remaining: 169ms
15:	learn: 0.6543808	total: 32.5ms	remaining: 170ms
16:	learn: 0.6510710	total: 34.5ms	remaining: 168ms
17:	learn: 0.6495314	total: 39.4ms	remaining: 179ms
18:	learn: 0.6468623	total: 40.9ms	remaining: 17

85:	learn: 0.5226180	total: 346ms	remaining: 56.3ms
86:	learn: 0.5209152	total: 349ms	remaining: 52.1ms
87:	learn: 0.5200760	total: 352ms	remaining: 48ms
88:	learn: 0.5181496	total: 354ms	remaining: 43.8ms
89:	learn: 0.5170079	total: 356ms	remaining: 39.6ms
90:	learn: 0.5149301	total: 358ms	remaining: 35.4ms
91:	learn: 0.5135335	total: 360ms	remaining: 31.3ms
92:	learn: 0.5117129	total: 362ms	remaining: 27.3ms
93:	learn: 0.5104164	total: 365ms	remaining: 23.3ms
94:	learn: 0.5086331	total: 368ms	remaining: 19.4ms
95:	learn: 0.5073898	total: 370ms	remaining: 15.4ms
96:	learn: 0.5056366	total: 372ms	remaining: 11.5ms
97:	learn: 0.5037133	total: 374ms	remaining: 7.63ms
98:	learn: 0.5029430	total: 375ms	remaining: 3.79ms
99:	learn: 0.5014492	total: 377ms	remaining: 0us
Learning rate set to 0.032107
0:	learn: 0.6901832	total: 1.86ms	remaining: 184ms
1:	learn: 0.6878122	total: 3.36ms	remaining: 165ms
2:	learn: 0.6848248	total: 4.86ms	remaining: 157ms
3:	learn: 0.6825513	total: 6.33ms	remainin

47:	learn: 0.5834424	total: 88.1ms	remaining: 95.5ms
48:	learn: 0.5796616	total: 90.2ms	remaining: 93.9ms
49:	learn: 0.5775608	total: 92.1ms	remaining: 92.1ms
50:	learn: 0.5768474	total: 93.6ms	remaining: 90ms
51:	learn: 0.5744274	total: 95.1ms	remaining: 87.8ms
52:	learn: 0.5715985	total: 96.6ms	remaining: 85.7ms
53:	learn: 0.5699280	total: 98.1ms	remaining: 83.6ms
54:	learn: 0.5675727	total: 99.6ms	remaining: 81.5ms
55:	learn: 0.5652188	total: 101ms	remaining: 79.4ms
56:	learn: 0.5639754	total: 103ms	remaining: 77.8ms
57:	learn: 0.5628551	total: 105ms	remaining: 76ms
58:	learn: 0.5611657	total: 107ms	remaining: 74.2ms
59:	learn: 0.5595284	total: 109ms	remaining: 72.3ms
60:	learn: 0.5578945	total: 110ms	remaining: 70.5ms
61:	learn: 0.5563695	total: 112ms	remaining: 68.7ms
62:	learn: 0.5546990	total: 114ms	remaining: 66.9ms
63:	learn: 0.5535429	total: 116ms	remaining: 65.1ms
64:	learn: 0.5513695	total: 118ms	remaining: 63.4ms
65:	learn: 0.5491156	total: 120ms	remaining: 61.6ms
66:	lear

32:	learn: 0.6104649	total: 55.4ms	remaining: 113ms
33:	learn: 0.6076340	total: 57.7ms	remaining: 112ms
34:	learn: 0.6049449	total: 59.7ms	remaining: 111ms
35:	learn: 0.6027615	total: 61.2ms	remaining: 109ms
36:	learn: 0.6006508	total: 62.7ms	remaining: 107ms
37:	learn: 0.5986022	total: 64.7ms	remaining: 106ms
38:	learn: 0.5964045	total: 66.5ms	remaining: 104ms
39:	learn: 0.5937707	total: 68.4ms	remaining: 103ms
40:	learn: 0.5913024	total: 70.4ms	remaining: 101ms
41:	learn: 0.5894833	total: 72.2ms	remaining: 99.8ms
42:	learn: 0.5878362	total: 74.1ms	remaining: 98.2ms
43:	learn: 0.5851062	total: 76ms	remaining: 96.7ms
44:	learn: 0.5827297	total: 78.2ms	remaining: 95.6ms
45:	learn: 0.5810107	total: 80ms	remaining: 93.9ms
46:	learn: 0.5793811	total: 81.9ms	remaining: 92.4ms
47:	learn: 0.5774021	total: 86.6ms	remaining: 93.9ms
48:	learn: 0.5747122	total: 88.5ms	remaining: 92.1ms
49:	learn: 0.5725759	total: 89.9ms	remaining: 89.9ms
50:	learn: 0.5706415	total: 91.4ms	remaining: 87.8ms
51:	le

Learning rate set to 0.032107
0:	learn: 0.6907389	total: 2.96ms	remaining: 293ms
1:	learn: 0.6876002	total: 5.02ms	remaining: 246ms
2:	learn: 0.6851382	total: 7.07ms	remaining: 229ms
3:	learn: 0.6822974	total: 9.31ms	remaining: 223ms
4:	learn: 0.6790159	total: 12ms	remaining: 227ms
5:	learn: 0.6762411	total: 14.5ms	remaining: 227ms
6:	learn: 0.6739351	total: 16.9ms	remaining: 224ms
7:	learn: 0.6716090	total: 22.1ms	remaining: 254ms
8:	learn: 0.6695345	total: 24ms	remaining: 242ms
9:	learn: 0.6672646	total: 26.2ms	remaining: 236ms
10:	learn: 0.6648971	total: 29.2ms	remaining: 237ms
11:	learn: 0.6614523	total: 32ms	remaining: 235ms
12:	learn: 0.6584001	total: 33.9ms	remaining: 227ms
13:	learn: 0.6560760	total: 35.6ms	remaining: 219ms
14:	learn: 0.6533393	total: 37.3ms	remaining: 211ms
15:	learn: 0.6512164	total: 39ms	remaining: 205ms
16:	learn: 0.6488396	total: 41.1ms	remaining: 201ms
17:	learn: 0.6456061	total: 42.8ms	remaining: 195ms
18:	learn: 0.6429268	total: 45.3ms	remaining: 193ms
